# Matrix Bloch Equation

### Basic Bloch Equation:
$$
\frac{d\vec{M}}{dt} = \vec{M} \times \gamma \vec{B} - \frac{\vec{M}_{xy}}{T_2} + \frac{M_0 - M_z}{T_1}
$$

### In Matrix Form:
Let:
$$
M =
\begin{bmatrix}
M_x \\
M_y \\
M_z
\end{bmatrix}
$$

(Usually, $M_0 = 1$)

### Becomes:
$$
\frac{dM}{dt} =
\begin{bmatrix}
-\frac{1}{T_2} & \gamma B_z & -\gamma B_y \\
-\gamma B_z & -\frac{1}{T_2} & \gamma B_x \\
\gamma B_y & -\gamma B_x & -\frac{1}{T_1}
\end{bmatrix}
M +
\begin{bmatrix}
0 \\
0 \\
\frac{M_0}{T_1}
\end{bmatrix}
$$


# Relaxation in Homogeneous Object and Uniform Field (Rot Frame)
### Over a Time Period $\tau$:
$$
M' =
\underbrace{
\begin{bmatrix}
E_2 & 0 & 0 \\
0 & E_2 & 0 \\
0 & 0 & E_1
\end{bmatrix}
}_{A}
M +
\underbrace{
\begin{bmatrix}
0 \\
0 \\
M_0(1 - E_1)
\end{bmatrix}
}_{B}
$$

### Definitions:
- $E_1 = e^{-\tau / T_1}$
- $E_2 = e^{-\tau / T_2}$

This equation describes the magnetization relaxation process over a time period $\tau$ with contributions from matrix $A$ (relaxation factors) and vector $B$ (equilibrium effects).


In [1]:
# %matplotlib inline
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider

def rotation_matrix(theta):
    return np.array([[np.cos(theta), np.sin(theta),0], [-np.sin(theta), np.cos(theta),0],[0,0,1]])

In [ ]:
def relaxation_rot(tau, T1, T2, M0=1):
    M = np.array([1, 0, 0]) 
    E1 = np.exp(-tau/T1)
    E2 = np.exp(-tau/T2)
    
    matrix = np.array([[E2, 0, 0], 
                        [0, E2, 0], 
                        [0, 0, E1]])
    
    A = matrix @ M
    B = np.array([0,0,M0*(1-E1)])
    M_star = A + B
    return M_star

relaxation_rot(0.5, 0.5, 0.1, 1)

In [ ]:

# Parameters
T1 = 1.0
T2 = 0.5


line = np.linspace(-1, 1, 100)  # x-axis for plotting

# Initial spin vector
initial_tau = 0
initial_spin = relaxation_rot(initial_tau, T1, T2)

# Create a 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(line, np.zeros(100), np.zeros(100), 'k--', lw=1)
ax.plot(np.zeros(100), line, np.zeros(100), 'k--', lw=1)
ax.plot(np.zeros(100), np.zeros(100), line, 'k--', lw=1)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Relaxation in the rotational frame')

# Plot Bloch sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z, color='b', alpha=0.1)

# Initial quiver (arrow)
quiver = ax.quiver(0, 0, 0, initial_spin[0], initial_spin[1], initial_spin[2], 
                   color='r', arrow_length_ratio=0.2, lw=2)

# Add a slider for tau
ax_slider = plt.axes([0.2, 0.02, 0.6, 0.03], facecolor='lightgoldenrodyellow')
slider = Slider(ax_slider, 'Time (tau)', 0, 3, valinit=initial_tau, valstep=0.1)

# Update function for the slider
def update(val):
    tau = slider.val
    spin_vector = relaxation_rot(tau, T1, T2)
    
    # Update the quiver by removing the old one and adding a new one
    global quiver
    quiver.remove()  # Remove the old quiver
    quiver = ax.quiver(0, 0, 0, spin_vector[0], spin_vector[1], spin_vector[2], 
                       color='r', arrow_length_ratio=0.2, lw=2)
    ax.scatter(spin_vector[0], spin_vector[1], spin_vector[2], color='r')
    fig.canvas.draw_idle()

# Connect the slider to the update function
slider.on_changed(update)

plt.show()

# Relaxation (Lab Frame)
See *"Principles of Magnetic Resonance"* of Dwight G. Nishimara p. 59. 
### Over a Time Period $\tau$:
$$
M' =
\underbrace{
\begin{bmatrix}
E_2 & 0 & 0 \\
0 & E_2 & 0 \\
0 & 0 & E_1
\end{bmatrix}
}_{A} 
Rot_z(\omega_0\tau)
M +
\underbrace{
\begin{bmatrix}
0 \\
0 \\
M_0(1 - E_1)
\end{bmatrix}
}_{B}
$$

### Definitions:
- $E_1 = e^{-\tau / T_1}$
- $E_2 = e^{-\tau / T_2}$

This equation describes the magnetization relaxation process over a time period $\tau$ with contributions from matrix $A$ (relaxation factors) and vector $B$ (equilibrium effects).


In [4]:
def relaxation_lab(tau, T1, T2, M0=1):
    M = np.array([1, 0, 0]) 
    E1 = np.exp(-tau/T1)
    E2 = np.exp(-tau/T2)
    R_z = rotation_matrix(8.4*tau)
    matrix = np.array([[E2, 0, 0], 
                        [0, E2, 0], 
                        [0, 0, E1]])
    
    A = matrix @ R_z @ M
    B = np.array([0,0,M0*(1-E1)])
    M_star = A + B
    return M_star

In [ ]:

# Parameters
T1 = 1.0
T2 = 0.5


line = np.linspace(-1, 1, 100)  # x-axis for plotting

# Initial spin vector
initial_tau = 0
initial_spin = relaxation_lab(initial_tau, T1, T2)

# Create a 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(line, np.zeros(100), np.zeros(100), 'k--', lw=1)
ax.plot(np.zeros(100), line, np.zeros(100), 'k--', lw=1)
ax.plot(np.zeros(100), np.zeros(100), line, 'k--', lw=1)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Relaxation in the lab frame')

# Plot Bloch sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z, color='b', alpha=0.1)

# Initial quiver (arrow)
quiver = ax.quiver(0, 0, 0, initial_spin[0], initial_spin[1], initial_spin[2], 
                   color='r', arrow_length_ratio=0.2, lw=2)

# Add a slider for tau
ax_slider = plt.axes([0.2, 0.02, 0.6, 0.03], facecolor='lightgoldenrodyellow')
slider = Slider(ax_slider, 'Time (tau)', 0, 3, valinit=initial_tau, valstep=0.1)

# Update function for the slider
def update(val):
    tau = slider.val
    spin_vector = relaxation_lab(tau, T1, T2)
    
    # Update the quiver by removing the old one and adding a new one
    global quiver
    quiver.remove()  # Remove the old quiver
    quiver = ax.quiver(0, 0, 0, spin_vector[0], spin_vector[1], spin_vector[2], 
                       color='r', arrow_length_ratio=0.2, lw=2)
    ax.scatter(spin_vector[0], spin_vector[1], spin_vector[2], color='r')
    fig.canvas.draw_idle()

# Connect the slider to the update function
slider.on_changed(update)

plt.show()